# Self-Driving Introduction
Derek Riley

This notebook allows a user to train a Deep Neural Network to "learn" how to drive a simulated car.  This was built by adapting a Udacity tutorial https://www.udacity.com/course/self-driving-car-engineer-nanodegree--nd013

Lets start with a few imports...

In [ ]:
import cv2
import csv
import numpy as np
import os

from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Conv2D, Cropping2D, Dropout, ELU
from keras.layers.pooling import MaxPooling2D
from keras.callbacks import ModelCheckpoint, Callback
from keras.regularizers import l2
from keras.optimizers import Adam

import sklearn
import matplotlib.pyplot as plt

import sys
import time
import argparse
import io
# %load_ext line_profiler

Next we define a function to create the Keras DNN.  Note that minor adjustments to this model can severly break it.  Please avoid the temptation to play around this until you have a solid understanding of the model and what constraints exist.  

In [ ]:
def commaAiModelPrime(time_len=1):
    """
    Creates comma.ai enhanced autonomous car  model
    Replaced dropout with regularization
    Added 3 additional convolution layers
    """
    model = Sequential()
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
    model.add(Cropping2D(cropping=((50,20), (0,0))))

    # Add three 5x5 convolution layers (output depth 64, and 64)
    model.add(Conv2D(16, (8, 8), strides=(4, 4), padding="same", kernel_regularizer=l2(0.001)))
    model.add(ELU())
    model.add(Conv2D(32, (5, 5), strides=(2, 2), padding="same", kernel_regularizer=l2(0.001)))
    model.add(ELU())
    model.add(Conv2D(48, (5, 5), strides=(2, 2), padding="same", kernel_regularizer=l2(0.001)))
    model.add(ELU())

    # Add two 3x3 convolution layers (output depth 64, and 64)
    model.add(Conv2D(64, (3, 3), padding='valid', kernel_regularizer=l2(0.001)))
    model.add(ELU())
    model.add(Conv2D(64, (3, 3), padding='valid', kernel_regularizer=l2(0.001)))
    model.add(ELU())

    model.add(Flatten())

    # model.add(Dropout(.2))
    model.add(Dense(100, kernel_regularizer=l2(0.001)))
    model.add(ELU())

    # model.add(Dropout(0.50))
    model.add(Dense(50, kernel_regularizer=l2(0.001)))
    model.add(ELU())

    # model.add(Dropout(0.50))
    model.add(Dense(10, kernel_regularizer=l2(0.001)))
    model.add(ELU())

    model.add(Dense(1))

    model.compile(optimizer=Adam(lr=1e-4), loss='mse')

    return model

The following functions load the driving data and images.  

In [ ]:
def getDrivingLogs(path, skipHeader=False):
    """
    Returns the lines from a driving log with base directory `dataPath`.
    If the file include headers, pass `skipHeader=True`.
    """
    lines = []
    with open(path + '/driving_log.csv') as csvFile:
        reader = csv.reader(csvFile)
        if skipHeader:
            next(reader, None)
        for line in reader:
            lines.append(line)
    return lines


def getImages(path):
    """
    Get all training images on the path `dataPath`.
    Returns `([centerPaths], [leftPath], [rightPath], [measurement])`
    """
    directories = [x[0] for x in os.walk(path)]
    dataDirectories = list(filter(lambda directory: os.path.isfile(
        directory + '/driving_log.csv'), directories))
    print(dataDirectories)
    centerTotal = []
    leftTotal = []
    rightTotal = []
    measurementTotal = []
    for directory in dataDirectories:
        lines = getDrivingLogs(directory, skipHeader=True)
        center = []
        left = []
        right = []
        measurements = []
        for line in lines:
            measurements.append(float(line[3]))
            center.append(directory + '/' + line[0].strip())
            left.append(directory + '/' + line[1].strip())
            right.append(directory + '/' + line[2].strip())
        centerTotal.extend(center)
        leftTotal.extend(left)
        rightTotal.extend(right)
        measurementTotal.extend(measurements)

    return (centerTotal, leftTotal, rightTotal, measurementTotal)

The following function combines images to prepare them to feed into the network.  

In [ ]:
def combineCenterLeftRightImages(center, left, right, measurement, correction):
    """
    Combine the image paths from `center`, `left` and `right` using the correction factor `correction`
    Returns ([imagePaths], [measurements])
    """
    imagePaths = []
    imagePaths.extend(center)
    imagePaths.extend(left)
    imagePaths.extend(right)
    measurements = []
    measurements.extend(measurement)
    measurements.extend([x + correction for x in measurement])
    measurements.extend([x - correction for x in measurement])
    return (imagePaths, measurements)

This generator provides data to train and validate the neural network.  

In [ ]:
def generator(samples, batch_size=32, flip = True):
    """
    Generate the required images and measurments for training/
    `samples` is a list of pairs (`imagePath`, `measurement`).
    """
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        samples = sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for imagePath, measurement in batch_samples:
                originalImage = cv2.imread(imagePath)
                image = cv2.cvtColor(originalImage, cv2.COLOR_BGR2RGB)
                images.append(image)
                angles.append(measurement)
                # Flipping
                if(flip):
                    images.append(cv2.flip(image,1))
                    angles.append(measurement*-1.0)

            # trim image to only see section with road
            inputs = np.array(images)
            outputs = np.array(angles)
            yield sklearn.utils.shuffle(inputs, outputs)

Below is the "main" functionality of this training.  You will need to adjust the data paths, and you will adjust the batch size and number of epochs.  

In [ ]:
my_home = "/home/harleys/CS-2300/lab8/SelfDrivingSim"
my_data = "/data/cs2300/L8"
my_batch_size = 32
my_epochs = 5

# Reading images locations.
centerPaths, leftPaths, rightPaths, measurements = getImages(my_data)
imagePaths, measurements = combineCenterLeftRightImages(
    centerPaths, leftPaths, rightPaths, measurements, 0.2)
print('Total Images: {}'.format(len(imagePaths)))

# Splitting samples and creating generators.
from sklearn.model_selection import train_test_split

samples = list(zip(imagePaths, measurements))
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

print('Train samples: {}'.format(len(train_samples)))
print('Validation samples: {}'.format(len(validation_samples)))

train_generator = generator(train_samples, batch_size=my_batch_size)
validation_generator = generator(validation_samples, batch_size=my_batch_size)

# Model creation
model = commaAiModelPrime()

# Train the model
history_object = model.fit_generator(train_generator, steps_per_epoch= \
    len(train_samples)/my_batch_size, validation_data=validation_generator, \
    validation_steps=len(validation_samples)/my_batch_size, epochs=my_epochs, verbose=1)

model.save(my_home + '/model_commaAiModelPrime_e5.h5')

print(history_object.history.keys())
print('Loss')
print(history_object.history['loss'])
print('Validation Loss')
print(history_object.history['val_loss'])

plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

In [ ]:
def create_model():
    my_home = "/home/harleys/CS-2300/lab8/SelfDrivingSim"
    my_data = "/home/harleys/CS-2300/lab8"
    my_batch_size = 32
    my_epochs = 20

    # Reading images locations.
    centerPaths, leftPaths, rightPaths, measurements = getImages(my_data)
    imagePaths, measurements = combineCenterLeftRightImages(
        centerPaths, leftPaths, rightPaths, measurements, 0.2)
    print('Total Images: {}'.format(len(imagePaths)))

    # Splitting samples and creating generators.
    from sklearn.model_selection import train_test_split

    samples = list(zip(imagePaths, measurements))
    train_samples, validation_samples = train_test_split(samples, test_size=0.2)

    print('Train samples: {}'.format(len(train_samples)))
    print('Validation samples: {}'.format(len(validation_samples)))

    train_generator = generator(train_samples, batch_size=my_batch_size)
    validation_generator = generator(validation_samples, batch_size=my_batch_size)

    # Model creation
    model = commaAiModelPrime()

    # Train the model
    history_object = model.fit_generator(train_generator, steps_per_epoch= \
        len(train_samples)/my_batch_size, validation_data=validation_generator, \
        validation_steps=len(validation_samples)/my_batch_size, epochs=my_epochs, verbose=1)

    model.save(my_home + '/final_model.h5')

    print(history_object.history.keys())
    print('Loss')
    print(history_object.history['loss'])
    print('Validation Loss')
    print(history_object.history['val_loss'])

    plt.plot(history_object.history['loss'])
    plt.plot(history_object.history['val_loss'])
    plt.title('model mean squared error loss')
    plt.ylabel('mean squared error loss')
    plt.xlabel('epoch')
    plt.legend(['training set', 'validation set'], loc='upper right')
    plt.show()

In [ ]:
# %lprun -f create_model create_model()